## Imports for the Project and Setup

In [1]:
# Full reset of the camera
!echo 'dlinano' | sudo -S systemctl restart nvargus-daemon && printf '\n'
# Check device number
#!ls -ltrh /dev/video*

[sudo] password for dlinano: 


In [21]:
import torch
import torchvision
from torchvision import transforms , datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary # for model summary
import matplotlib.pyplot as plt # for ploting our data and showing images
import numpy as np
import cv2
import dataHelper as dH
import os
import time
import ipywidgets
from IPython.display import display, Javascript
from jupyter_clickable_image_widget import ClickableImageWidget
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
from jetcam.csi_camera import CSICamera
%matplotlib inline

In [3]:
camera = CSICamera(width=224, height=224)

## Model Setup

Here we are letting pytorch know to use the gpu and we are also importing a pretrained resnet18 model that we are modifying to return only 2 outputs. One for X and one for Y.

In [4]:
device = torch.device('cuda')
output_dim = 2
# RESNET 18
model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, output_dim)
model = model.to(device)

## Model Summary
This is the inner workings of resnet18.

In [5]:
summary(model, input_size=(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

## Data Collection

### Files Structure Setup

In [6]:
path_raw = os.getcwd() + '/raw_datasets'
try:
    os.mkdir(path_raw)
except OSError:
    pass
else:
    pass

In [8]:
#config
datacollect = True
dataSetSize = 5
FPScollect = 30
SEC_delay = 2


loopcounter = 0
collectedNum = 0

if datacollect == True:
    image_widget = ipywidgets.Image(format='jpeg')
    
    camera.unobserve_all()
    print('Get Ready!')
    frame = camera.read()
    image_widget.value = bgr8_to_jpeg(frame)
    #plt.imshow(bgr8_to_jpeg(frame))
    time.sleep(6)
    display(image_widget)
    print('Begin!')
    
    while True:
        if collectedNum == dataSetSize:
            break
        frame = camera.read()
        #plt.imshow(frame)
        image_widget.value = bgr8_to_jpeg(frame)
        if loopcounter % (FPScollect * SEC_delay) == 0:
            # write raw images to file 
            dH.write_raw_frame(path_raw, frame, collectedNum)
            collectedNum += 1
        loopcounter += 1
print('Done!')

Get Ready!


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Begin!
Done!


In [17]:
path_labeled = os.getcwd() + '/labeled_datasets'
try:
    os.mkdir(path_labeled)
except OSError:
    pass
else:
    pass

def on_click(_, content,msg):
        if content['event'] == 'click':
            data = content['eventData']
            x = data['offsetX']
            y = data['offsetY']
            print(f'{x},{y}')


In [24]:
datalabel = True


if datalabel == True:
    
    #click_widget = ClickableImageWidget(width=224,height=224)
    #click_widget.on_msg(on_click)
    #display(click_widget)
    
    def on_image_click():
        #do something....
        return 
    
    with open(path_raw + '/' +'img_raw.0.jpg', "rb") as f:
        image = f.read()
    
    with open(path_raw + '/' +'img_raw.1.jpg', "rb") as f:
        image = f.read()
        w = ipywidgets.Image(
            value=image,
            width=224,
            height=224)
        w.add_class('the_image_class')
    
    #Now, I wrote some javascript(jQuery) code like this...
    js = '''$(".the_image_class").on("click", function(e){
             var kernel = IPython.notebook.kernel;
             kernel.execute("on_image_click()");
          });'''
    #then, run the javascript...
    display(Javascript(js))
    
    
    
    #click_widget.on_msg(on_click)
    #display(ipywidgets.HBox([w]))
    display(w)
    #display(click_widget)

<IPython.core.display.Javascript object>

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…